In [105]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit
import matplotlib.pyplot as plt
import scipy.stats as stats
import openpyxl as xl


In [106]:
Macro = pd.read_excel("macro.xlsx",engine='openpyxl')
Macro["Date"] = pd.to_datetime(Macro["Date"])
Macro = Macro.sort_values(by=["Date"],ascending=True)
Macro.head()


,Date,ffr,Mkt-RF,RF,us10y
0,1995-01-01,4.94,NaN,NaN,NaN
1,1995-01-02,4.94,NaN,NaN,NaN
2,1995-01-03,5.98,-0.26,0.02,7.88
3,1995-01-04,6.59,0.32,0.02,7.82
4,1995-01-05,5.72,-0.05,0.02,7.88


11

In [107]:
avgMkt = Macro["Mkt-RF"].mean()
print(avgMkt)

0.03913119612068966


12

In [108]:
lowffr = Macro["ffr"].min()
print(lowffr)

0.04


13

In [109]:
max_us10y_pct = Macro["us10y"].max()
print(max_us10y_pct)

7.89


14

In [110]:
stock = pd.read_excel("stock.xlsx",engine='openpyxl')
stock["Date"] = pd.to_datetime(stock["Date"])
stock = stock.sort_values(by=["Date"],ascending=True)
stock.head()

,Date,Close
0,1995-01-03,15.15625
1,1995-01-04,15.21875
2,1995-01-05,15.25000
3,1995-01-06,15.25000
4,1995-01-09,15.12500


In [111]:
avgstock = stock["Close"].mean()
print(avgstock)

63.86271873046874


15

In [112]:
stock["stock_return_pct"] = stock["Close"].pct_change() * 100
avg_stock_return_pct = stock["stock_return_pct"].mean()
print(avg_stock_return_pct)

0.04073094094750926


16

In [113]:
mps = pd.read_excel("mps.xlsx",engine='openpyxl')
mps["Date"] = pd.to_datetime(mps["Date"])
mps = mps.sort_values(by=["Date"],ascending=True)
mps.head()

,Date,mps,FOMC
0,1995-02-01,0.041877,1
1,1995-03-28,0.028769,1
2,1995-05-23,0.000864,1
3,1995-07-06,-0.151026,1
4,1995-08-22,0.035412,1


In [114]:
suprise = mps["mps"].notna().sum()
print(suprise)

234


In [115]:
newdf = stock
newdf = stock.merge(Macro, on="Date", how="left")
newdf = newdf.merge(mps, on="Date", how="left")

newdf = newdf.sort_values("Date")

newdf["ffr_lag"] = newdf["ffr"].shift(1)
newdf["Mkt-RF_lag"] = newdf["Mkt-RF"].shift(1)
newdf["us10y_lag"] = newdf["us10y"].shift(1)

newdf["mps"] = newdf["mps"].fillna(0)
newdf["FOMC"] = newdf["FOMC"].fillna(0)

newdf["excess_return"] = newdf["stock_return_pct"] - newdf["RF"]
newdf = newdf.dropna().reset_index(drop=True)


avg_excess = newdf["excess_return"].mean()
print(avg_excess)

0.029386352727986127


18

In [116]:
Y = newdf["excess_return"]
X = newdf[["mps", "Mkt-RF", "ffr_lag", "us10y_lag"]]
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()

p_value_mps = model.pvalues["mps"]
p_value_mps

np.float64(0.3846738249461573)

19

In [117]:
r_squared = model.rsquared
r_squared

np.float64(0.3311097438471893)

20

In [118]:
df20 = newdf[["Date", "FOMC", "us10y", "us10y_lag"]].copy()
df20["delta_us10y"] = df20["us10y"] - df20["us10y_lag"]
df20 = df20.dropna()

Y = df20["delta_us10y"]
X = sm.add_constant(df20["FOMC"])

model_20 = sm.OLS(Y, X).fit()
model_20.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            delta_us10y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     4.624
Date:                Sat, 10 Jan 2026   Prob (F-statistic):             0.0316
Time:                        12:35:48   Log-Likelihood:                 10396.
No. Observations:                7311   AIC:                        -2.079e+04
Df Residuals:                    7309   BIC:                        -2.077e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002      0.001     -0.324      0.746      -0.002       0.001
FOMC          -0.0084      0.004     -2.150      0.032      -0.016      -0.001
==============================================================================
Omnibus:                      424.848   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1812.120
Skew:                           0.045   Prob(JB):                         0.00
Kurtosis:                       5.437   Cond. No.                         5.70
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [119]:
print(model_20.params["FOMC"])


-0.008359051258387245


In [120]:
print(model_20.pvalues["FOMC"])

0.03155472216351596
